In [67]:
import google.generativeai as genai
import textwrap
import pandas as pd
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI
import datetime 
from typing import Literal, Optional, Tuple 
from langchain_core.pydantic_v1 import BaseModel, Field 
from langchain.output_parsers import PydanticToolsParser 
from langchain_core.prompts import ChatPromptTemplate 
from langchain_openai import ChatOpenAI 
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
import sqlite3



load_dotenv()
API_KEY = os.getenv("API_KEY")
nlpfpath='Intern NLP Dataset.xlsx'
os.environ["OPENAI_API_KEY"] = API_KEY

In [68]:
df=pd.read_excel(nlpfpath)
# df.head()
columns=df.columns.tolist()
columns
sample=df.head().to_string()

In [70]:

class SubQuery(BaseModel): 
    """Search over a database of Visual Attribute Time Series Dataset""" 
    sub_query: str = Field(description="A very specific query against the database.", 
    )

system = f"""You are an expert at converting user questions into database 
queries. \ 
You have access to a database of Visual Attribute Time Series Dataset .  \ 
Perform query decomposition. Given a user question, break it down into 
distinct sub questions that \ 
you need to answer in order to answer the original question. 
If there are acronyms or words you are not familiar with, do not try to 
rephrase them.""" 
prompt = ChatPromptTemplate.from_messages( 
[ 
("system", system), 
("human", "{question}"), 
] 
) 
llm = ChatOpenAI(model="gpt-4o", 
temperature=0) 
llm_with_tools = llm.bind_tools([SubQuery]) 
parser = PydanticToolsParser(tools=[SubQuery]) 
query_analyzer = prompt | llm_with_tools | parser 
subqueries=query_analyzer.invoke({"question": "Which minute did I get the most visitors, and Who is my most common visitor?"}) 


In [71]:
response_schemas = [
    ResponseSchema(name="query", description="structured query to be executed against the sql lite3."),
    ResponseSchema(
        name="description",
        description="the description of the result the names of selected columns",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
template = PromptTemplate(
    template="answer the users question: {question}\n as database query on this database with this head sample {dataset} and table name nlp: .\n{format_instructions}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions,"dataset":sample},
)
print(prompt)
chain=prompt|llm|output_parser
# while (True):
#    input("") 
# for subquery in subqueries: 

#     print(subquery.sub_query)
#     print(chain.invoke({"question": subquery.sub_query}))


input_variables=['question'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an expert at converting user questions into database \nqueries. \\ \nYou have access to a database of Visual Attribute Time Series Dataset .  \\ \nPerform query decomposition. Given a user question, break it down into \ndistinct sub questions that \\ \nyou need to answer in order to answer the original question. \nIf there are acronyms or words you are not familiar with, do not try to \nrephrase them.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]


In [72]:
def read_sql_query(query,db):
    conn=sqlite3.connect(db)
    cur=conn.cursor()
    cur.execute(query)
    rows=cur.fetchall()
    conn.commit()
    conn.close()
    # for row in rows:
    #     # pass
    #     print(row)
    return rows

In [73]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage 
firstsubquery="Which minute did I get the most visitors"
if __name__=='__main__':
    print("here is a sample from the dataset ")
    print(sample)
    question=input("Enter the quesion you want to ask the database")
    quitt=None
    while quitt!='q':
        
        subqueries=query_analyzer.invoke({"question": question})
        print("the subqueries are: ",subqueries)
        baseconversaction=[
            SystemMessage(content="You are an expert at converting user questions into database queries")
        ]
        for subquery in subqueries: 
            humanformat=template.format(question=subquery)
            baseconversaction.append(HumanMessage(content=humanformat))
            airesponse=llm.invoke(baseconversaction)
            baseconversaction.append(AIMessage(content=airesponse.content))
            # print("the ai reponse is: ",airesponse.content)
            parsedoutput=output_parser.parse(airesponse.content)
            # print("parsed output is: ",parsedoutput)
            query=parsedoutput['query']
            # print("the query is: ",query)
            for i in range(10):
                try:
                    if i==9:
                        raise Exception("The query failed after 10 retries rewrite your prompt")
                    rows=read_sql_query(query,'nlp.db')
                    print(f"the answer for this {parsedoutput['description']} is: ",rows)
                    break
                except Exception as e:
                    humanerror=f"the query {query} failed with error {e} rewrite it"
                    baseconversaction.append(HumanMessage(content=humanerror))
                    airesponse=llm.invoke(baseconversaction)
                    baseconversaction.append(AIMessage(content=airesponse.content))
                    # print(airesponse.content)
                    parsedoutput=output_parser.parse(airesponse.content)
                    # print(parsedoutput)
                    query=parsedoutput['query']
                    # print("the query is: ",query)
                    continue
        quitt=input("write q to quit or any other key to continue")
        if quitt!='q':

            question=input("Reenter quesion or rewrite it in differnt format")
        
        



here is a sample from the dataset 
                     Time  Is Male  Is Female  Is Hijab  Is Child  Is Niqab  Has Bag  Cluster ID
0 2024-07-17 14:30:38.456        0          1         1         0         0        0          35
1 2024-07-17 14:31:41.128        0          1         0         0         1        0          35
2 2024-07-17 14:33:14.869        0          1         1         0         0        0          35
3 2024-07-17 14:33:16.417        0          1         1         0         0        0          36
4 2024-07-17 14:33:18.821        1          0         0         0         0        0          36
the subqueries are:  [SubQuery(sub_query='Which minute did I get the most visitors?'), SubQuery(sub_query='Who is my most common visitor?')]
the answer for this The minute and the number of visitors during that minute when you got the most visitors. is:  [('2024-07-17 15:12', 5)]
the answer for this The most common visitor profile based on gender, hijab, child, niqab, and bag attr

creating the db

In [ ]:


df=pd.read_excel(nlpfpath)
df.head()
#  convert this file to sql file
import sqlite3
conn = sqlite3.connect('nlp.db')
c = conn.cursor()
df.to_sql('nlp', conn, if_exists='replace', index = False)
conn.commit()



# GOOGLE GEMINI TEST

In [ ]:


prompt = f"""
You are an expert in converting English questions to SQL query!
The SQL database has the name nlp and has the following columns 
- {columns}

For example,
Example 1 - How many entries of records are present?, 
the SQL command will be something like this: SELECT COUNT(*) FROM nlp;

also the SQL code should not have ``` 
in the beginning or end and SQL word in output
"""


print(prompt)

def get_gemini_response(question,prompt):
    model=genai.GenerativeModel('gemini-pro')
    response=model.generate_content([prompt,question])
    return response.text
# print(query)
while True:
    try:
        # question='Which minute did I get the most visitors, and Who is my most common visitor?'
        question= 'and Who is my most common visitor?'
        query=get_gemini_response(question,prompt)
        print(query)
        result=read_sql_query(query,'nlp.db')
        # print(result)
        break

    except Exception as e:
        print(e)
        ##add the error to the prompt[0]
        print
        prompt[0]=prompt[0]+"repeat the process to fix this error"+ "here is the query: " +query+"here is the error: " +str(e)



You are an expert in converting English questions to SQL query!
The SQL database has the name nlp and has the following columns 
- ['Time', 'Is Male', 'Is Female', 'Is Hijab', 'Is Child', 'Is Niqab', 'Has Bag', 'Cluster ID']

For example,
Example 1 - How many entries of records are present?, 
the SQL command will be something like this: SELECT COUNT(*) FROM nlp;

also the SQL code should not have ``` 
in the beginning or end and SQL word in output


  No API_KEY or ADC found. Please either:
    - Set the `GOOGLE_API_KEY` environment variable.
    - Manually pass the key with `genai.configure(api_key=my_api_key)`.
    - Or set up Application Default Credentials, see https://ai.google.dev/gemini-api/docs/oauth for more information.


TypeError: 'str' object does not support item assignment

In [ ]:
select = genai.protos.Schema(
        type=genai.protos.Type.STRING,
    )
selectcols = genai.protos.Schema(
    type=genai.protos.Type.ARRAY,
    items=select
)
where = genai.protos.Schema(
        type=genai.protos.Type.STRING,
    )
wherecols = genai.protos.Schema(
    type=genai.protos.Type.ARRAY,
    items=where
)
group = genai.protos.Schema(
        type=genai.protos.Type.STRING,
    )
groupcols = genai.protos.Schema(
    type=genai.protos.Type.ARRAY,
    items=group
)
order = genai.protos.Schema(
        type=genai.protos.Type.STRING,
    )
ordercols = genai.protos.Schema(
    type=genai.protos.Type.ARRAY,
    items=order
)
queryobj = genai.protos.FunctionDeclaration(
name="structure_query",
description=textwrap.dedent("""\
    structure a query from the given text
    """),
parameters=genai.protos.Schema(
    type=genai.protos.Type.OBJECT,
    properties = {
        'selectcols': selectcols,
        'wherecols':wherecols,
        'groupcols':groupcols,
        'ordercols':ordercols,
        
        
    }
)
)

In [ ]:
print(queryobj)

name: "structure_query"
description: "structure a query from the given text\n"
parameters {
  type_: OBJECT
  properties {
    key: "wherecols"
    value {
      type_: ARRAY
      items {
        type_: STRING
      }
    }
  }
  properties {
    key: "selectcols"
    value {
      type_: ARRAY
      items {
        type_: STRING
      }
    }
  }
  properties {
    key: "ordercols"
    value {
      type_: ARRAY
      items {
        type_: STRING
      }
    }
  }
  properties {
    key: "groupcols"
    value {
      type_: ARRAY
      items {
        type_: STRING
      }
    }
  }
}



In [ ]:
model = model = genai.GenerativeModel(
    model_name='models/gemini-1.5-pro-latest',
    tools = [queryobj])

['Time',
 'Is Male',
 'Is Female',
 'Is Hijab',
 'Is Child',
 'Is Niqab',
 'Has Bag',
 'Cluster ID']

In [ ]:
#get the data types
data_types = df.dtypes
data_types

Time          datetime64[ns]
Is Male                int64
Is Female              int64
Is Hijab               int64
Is Child               int64
Is Niqab               int64
Has Bag                int64
Cluster ID             int64
dtype: object

In [ ]:
query='Which minute did I get the most visitors, and Who is my most common visitor?'
modelinput=f"""
Please structure this query: {query} on these columns: {columns} where the values are binary except for time and cluster ID
"""
print(modelinput)
result = model.generate_content(
    modelinput
,tool_config={'function_calling_config':'ANY'})    
fc=result.candidates[0].content.parts[0].function_call
fc=type(fc).to_dict(fc)
fc


Please structure this query: Which minute did I get the most visitors, and Who is my most common visitor? on these columns: ['Time', 'Is Male', 'Is Female', 'Is Hijab', 'Is Child', 'Is Niqab', 'Has Bag', 'Cluster ID'] where the values are binary except for time and cluster ID



{'name': 'structure_query',
 'args': {'selectcols': ['Time', 'count(*)'],
  'wherecols': ['Is Visitor  =  1'],
  'groupcols': ['Time'],
  'ordercols': ['count(*) DESC']}}

In [ ]:
# df.query(fc['args']['selectcols'][0])

KeyError: "None of [DatetimeIndex(['2024-07-17 14:30:38.456000', '2024-07-17 14:31:41.128000',\n               '2024-07-17 14:33:14.869000', '2024-07-17 14:33:16.417000',\n               '2024-07-17 14:33:18.821000', '2024-07-17 14:33:21.427000',\n               '2024-07-17 14:34:04.205000', '2024-07-17 14:36:01.130000',\n               '2024-07-17 14:36:40.263000', '2024-07-17 14:37:37.979000',\n               '2024-07-17 14:37:40.097000', '2024-07-17 14:40:41.511000',\n               '2024-07-17 14:42:56.267000', '2024-07-17 14:42:56.521000',\n               '2024-07-17 14:43:37.136000', '2024-07-17 14:44:17.314000',\n               '2024-07-17 14:45:07.718000', '2024-07-17 14:45:16.084000',\n               '2024-07-17 14:45:17.967000', '2024-07-17 14:46:05.075000',\n               '2024-07-17 14:46:06.585000', '2024-07-17 14:46:08.832000',\n               '2024-07-17 14:49:14.912000', '2024-07-17 14:50:55.856000',\n               '2024-07-17 14:50:56.117000', '2024-07-17 14:51:13.52